In [132]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import MySQLdb
import datetime
from sklearn.metrics import auc, mean_squared_error

In [115]:
import codecs
import lxml
from lxml import etree
import os
from mmap import mmap
import re
from os import listdir
from os.path import isfile, join
import MySQLdb
from datetime import datetime

In [67]:
import matplotlib as mpl
import datetime

## Запишем в базу

In [170]:
whole_res = pd.read_csv('fpk_data/model/4ypred.csv', sep=';')

In [198]:
def proccess_flight(r):
    cur.execute("INSERT INTO Forecast(fltNr, opsSuffix, Code, SegOrig, SegDest, TicketActual, TicketForecast, DptDate, reportDt) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                (int(r[1][whole_res.columns.get_loc("fltNr")]), r[1][whole_res.columns.get_loc("opsSuffix")], r[1][whole_res.columns.get_loc("Code")],
                 r[1][whole_res.columns.get_loc("SegOrig")], r[1][whole_res.columns.get_loc("SegDest")], r[1][whole_res.columns.get_loc("actual")], 
                 r[1][whole_res.columns.get_loc("predictions")], r[1][whole_res.columns.get_loc("fltNrDptDt")], ts))

In [199]:
# fltNr
flightNrSf_dict = {0: '10J', 1: '50M', 2: '9J', 3: '49J', 4: '337J', 5: '337ZH', 6: '138J', 7: '137YQ', 8: '102J', 9: '101J', 10: '88J', 11: '87J'}
whole_res['flightNrSf'] = whole_res['flightNrSf'].map(lambda x: flightNrSf_dict.get(x))
def map_x(x):
    if x == '337ZH' or x == '137YQ':
        return x[:-2]
    else:
        return x[:-1]
        
whole_res['fltNr'] = whole_res['flightNrSf'].map(lambda x: map_x(x))
def map_x(x):
    if x == '337ZH' or x == '137YQ':
        return x[-2:]
    else:
        return x[-1:]

whole_res['opsSuffix'] = whole_res['flightNrSf'].map(lambda x: map_x(x))

# seg
segment_dict = {0: '12_23', 1: '12_25', 2: '12_58', 3: '12_13', 4: '23_25', 5: '23_58', 6: '23_13', 7: '25_58', 8: '25_13', 9: '58_13', 10: '12_24', 11: '12_28', 12: '12_16', 13: '12_2', 14: '12_0', 15: '12_41', 16: '23_24', 17: '23_28', 18: '23_16', 19: '23_2', 20: '23_0', 21: '23_41', 22: '24_28', 23: '24_16', 24: '24_25', 25: '24_2', 26: '24_0', 27: '24_58', 28: '24_41', 29: '24_13', 30: '28_16', 31: '28_25', 32: '28_2', 33: '28_0', 34: '28_58', 35: '28_41', 36: '28_13', 37: '13_25', 38: '13_23', 39: '13_12', 40: '25_23', 41: '25_12', 42: '23_12', 43: '13_54', 44: '13_16', 45: '13_28', 46: '13_24', 47: '54_25', 48: '54_16', 49: '54_28', 50: '54_24', 51: '54_23', 52: '54_12', 53: '25_16', 54: '25_28', 55: '25_24', 56: '16_28', 57: '16_24', 58: '16_23', 59: '16_12', 60: '28_24', 61: '28_23', 62: '28_12', 63: '24_23', 64: '24_12', 65: '13_58', 66: '13_2', 67: '13_1', 68: '13_8', 69: '13_20', 70: '13_48', 71: '13_62', 72: '13_52', 73: '13_9', 74: '13_15', 75: '13_11', 76: '13_55', 77: '13_45', 78: '13_31', 79: '13_32', 80: '13_38', 81: '13_60', 82: '58_2', 83: '58_25', 84: '58_1', 85: '58_8', 86: '58_20', 87: '58_48', 88: '58_62', 89: '58_52', 90: '58_9', 91: '58_15', 92: '58_11', 93: '58_55', 94: '58_45', 95: '58_31', 96: '58_32', 97: '58_38', 98: '58_60', 99: '2_25', 100: '2_1', 101: '2_8', 102: '60_38', 103: '60_31', 104: '60_45', 105: '60_55', 106: '60_11', 107: '60_15', 108: '60_9', 109: '60_52', 110: '60_62', 111: '60_48', 112: '60_20', 113: '60_8', 114: '60_1', 115: '60_25', 116: '60_2', 117: '60_54', 118: '60_58', 119: '60_13', 120: '38_31', 121: '38_45', 122: '38_55', 123: '38_11', 124: '38_15', 125: '38_9', 126: '38_52', 127: '38_62', 128: '38_48', 129: '12_21', 130: '12_10', 131: '12_3', 132: '12_34', 133: '12_56', 134: '12_53', 135: '12_27', 136: '21_24', 137: '21_10', 138: '21_16', 139: '21_3', 140: '21_25', 141: '21_34', 142: '21_56', 143: '21_2', 144: '21_53', 145: '21_58', 146: '21_27', 147: '21_41', 148: '21_13', 149: '24_10', 150: '24_3', 151: '24_34', 152: '24_56', 153: '24_53', 154: '24_27', 155: '10_16', 156: '10_3', 157: '10_25', 158: '10_34', 159: '16_25', 160: '16_2', 161: '12_51', 162: '13_34', 163: '16_0', 164: '16_58', 165: '16_41', 166: '16_13', 167: '25_2', 168: '25_0', 169: '25_41', 170: '2_0', 171: '2_58', 172: '2_41', 173: '2_13', 174: '0_58', 175: '0_41', 176: '0_13', 177: '58_41', 178: '41_13', 179: '13_21', 180: '58_34', 181: '58_16', 182: '58_21', 183: '58_23', 184: '58_12', 185: '34_25', 186: '34_16', 187: '34_21', 188: '34_23', 189: '34_12', 190: '25_21', 191: '16_21', 192: '21_23', 193: '21_12', 194: '38_20', 195: '38_8', 196: '21_28', 197: '21_51', 198: '12_33', 199: '38_1', 200: '21_33', 201: '38_25', 202: '38_2', 203: '38_54', 204: '38_58', 205: '38_13', 206: '31_45', 207: '31_55', 208: '31_11', 209: '31_15', 210: '24_51', 211: '60_4', 212: '31_4', 213: '31_52', 214: '31_62', 215: '31_48', 216: '31_20', 217: '31_1', 218: '31_25', 219: '31_2', 220: '31_58', 221: '31_13', 222: '11_15', 223: '42_58', 224: '42_13', 225: '42_44', 226: '42_5', 227: '42_14', 228: '42_39', 229: '42_40', 230: '42_26', 231: '42_37', 232: '42_17', 233: '42_35', 234: '42_43', 235: '58_44', 236: '58_5', 237: '58_14', 238: '58_39', 239: '58_40', 240: '58_26', 241: '58_37', 242: '58_17', 243: '58_35', 244: '58_43', 245: '13_44', 246: '13_5', 247: '13_14', 248: '13_39', 249: '13_40', 250: '13_26', 251: '13_37', 252: '13_17', 253: '13_35', 254: '13_43', 255: '44_5', 256: '44_14', 257: '44_39', 258: '44_40', 259: '44_26', 260: '44_37', 261: '44_17', 262: '44_35', 263: '44_43', 264: '5_14', 265: '5_39', 266: '5_40', 267: '5_26', 268: '5_37', 269: '5_17', 270: '5_35', 271: '5_43', 272: '14_39', 273: '14_40', 274: '14_26', 275: '14_37', 276: '14_17', 277: '14_35', 278: '14_43', 279: '25_1', 280: '25_20', 281: '25_48', 282: '25_62', 283: '25_52', 284: '25_9', 285: '25_15', 286: '25_11', 287: '25_31', 288: '25_32', 289: '25_38', 290: '25_60', 291: '1_20', 292: '1_48', 293: '1_62', 294: '1_52', 295: '1_9', 296: '1_15', 297: '28_10', 298: '10_2', 299: '10_58', 300: '10_13', 301: '11_52', 302: '11_62', 303: '11_48', 304: '11_20', 305: '11_1', 306: '11_25', 307: '11_2', 308: '11_58', 309: '11_13', 310: '15_52', 311: '15_62', 312: '15_48', 313: '15_20', 314: '15_1', 315: '15_25', 316: '15_2', 317: '2_20', 318: '2_48', 319: '2_62', 320: '2_52', 321: '2_15', 322: '43_35', 323: '43_17', 324: '43_37', 325: '43_26', 326: '43_40', 327: '43_14', 328: '43_5', 329: '43_44', 330: '43_13', 331: '43_58', 332: '43_57', 333: '43_42', 334: '35_17', 335: '15_58', 336: '15_13', 337: '52_62', 338: '52_48', 339: '52_20', 340: '52_1', 341: '52_25', 342: '52_2', 343: '2_11', 344: '2_31', 345: '2_32', 346: '2_60', 347: '43_39', 348: '35_37', 349: '35_26', 350: '35_40', 351: '35_39', 352: '35_14', 353: '35_5', 354: '35_44', 355: '35_13', 356: '35_58', 357: '35_57', 358: '35_42', 359: '17_37', 360: '17_26', 361: '17_40', 362: '17_39', 363: '17_14', 364: '17_5', 365: '17_44', 366: '17_13', 367: '17_58', 368: '17_57', 369: '17_42', 370: '37_26', 371: '37_40', 372: '37_39', 373: '37_14', 374: '37_5', 375: '37_44', 376: '37_13', 377: '37_58', 378: '37_57', 379: '37_42', 380: '26_40', 381: '26_39', 382: '26_14', 383: '26_5', 384: '26_44', 385: '26_13', 386: '26_58', 387: '26_57', 388: '26_42', 389: '40_39', 390: '40_14', 391: '40_5', 392: '40_44', 393: '11_4', 394: '39_40', 395: '39_26', 396: '39_37', 397: '39_17', 398: '39_35', 399: '39_43', 400: '40_26', 401: '40_37', 402: '40_17', 403: '40_35', 404: '40_43', 405: '26_37', 406: '26_17', 407: '26_35', 408: '26_43', 409: '37_17', 410: '37_35', 411: '37_43', 412: '17_35', 413: '17_43', 414: '35_43', 415: '13_29', 416: '44_29', 417: '5_29', 418: '14_29', 419: '40_29', 420: '10_0', 421: '2_16', 422: '2_28', 423: '2_24', 424: '2_23', 425: '2_12', 426: '31_9', 427: '11_9', 428: '40_13', 429: '39_14', 430: '39_5', 431: '39_44', 432: '39_13', 433: '14_5', 434: '14_44', 435: '14_13', 436: '5_44', 437: '5_13', 438: '40_58', 439: '15_4', 440: '44_13', 441: '52_58', 442: '52_13', 443: '62_48', 444: '62_20', 445: '62_1', 446: '40_57', 447: '31_8', 448: '31_54', 449: '45_55', 450: '45_11', 451: '45_15', 452: '45_9', 453: '45_52', 454: '45_62', 455: '40_42', 456: '12_36', 457: '23_34', 458: '23_36', 459: '24_36', 460: '25_34', 461: '25_36', 462: '34_58', 463: '34_36', 464: '34_13', 465: '58_36', 466: '36_13', 467: '13_51', 468: '54_51', 469: '25_51', 470: '16_51', 471: '51_24', 472: '51_23', 473: '51_12', 474: '23_21', 475: '12_18', 476: '12_46', 477: '12_47', 478: '12_30', 479: '18_46', 480: '18_47', 481: '18_30', 482: '46_47', 483: '46_30', 484: '47_30', 485: '13_22', 486: '13_41', 487: '13_27', 488: '60_32', 489: '60_59', 490: '60_6', 491: '60_61', 492: '60_50', 493: '12_49', 494: '12_19', 495: '12_7', 496: '18_49', 497: '18_19', 498: '18_7', 499: '49_19', 500: '49_46', 501: '49_7', 502: '49_47', 503: '49_30', 504: '19_46', 505: '19_7', 506: '19_47', 507: '19_30', 508: '46_7', 509: '7_47', 510: '7_30'}
whole_res['segment'] = whole_res['segment'].map(lambda x: segment_dict.get(x))
whole_res['SegOrig'] = whole_res['segment'].map(lambda x: x.split('_')[0])
whole_res['SegDest'] = whole_res['segment'].map(lambda x: x.split('_')[1])
segment_dict = {0: 'BARYS', 1: 'SKX', 2: 'INZA', 3: 'WINOP', 4: 'YTNPR', 5: 'AZOQM', 6: 'NEKOZ', 7: 'URMAR', 8: 'UZHOV', 9: 'NEREK', 10: 'TORBV', 11: 'RYBP', 12: 'MOWKZ', 13: 'KUF', 14: 'CEK', 15: 'IARGL', 16: 'KOVYL', 17: 'PYTJA', 18: 'VEKVK', 19: 'NAVAS', 20: 'ARZM1', 21: 'SHILV', 22: 'NOVKU', 23: 'RZN1', 24: 'SASOV', 25: 'RUZVK', 26: 'TJM', 27: 'BEZNC', 28: 'ZUBVP', 29: 'LANGP', 30: 'KZNP', 31: 'BOLOG', 32: 'OKUL', 33: 'VOEVD', 34: 'GFBMH', 35: 'SGC', 36: 'OBSHA', 37: 'TOX', 38: 'MVISH', 39: 'KMNUR', 40: 'SVX', 41: 'CHAPV', 42: 'PEZ1', 43: 'NJC', 44: 'SXMBV', 45: 'MAKSA', 46: 'KANAS', 47: 'ZELDO', 48: 'GOJ', 49: 'MUROM', 50: 'VOLGA', 51: 'POTMA', 52: 'IVANV', 53: 'BAZRN', 54: 'KUZOV', 55: 'SONKV', 56: 'NOCKA', 57: 'KUZNT', 58: 'SYZR1', 59: 'BEZHK', 60: 'LEDGL', 61: 'SHSTH', 62: 'KOVRV'}
whole_res['SegOrig'] = whole_res['SegOrig'].map(lambda x: segment_dict.get(int(x)))
whole_res['SegDest'] = whole_res['SegDest'].map(lambda x: segment_dict.get(int(x)))

# code
code_dict = {0: 'F', 1: 'C', 2: 'B', 3: 'Y', 4: 'X', 5: 'K'}
whole_res['Code'] = whole_res['Code'].map(lambda x: code_dict.get(x))

In [119]:
whole_res

,FlightID,Code,fltNrDptDt,DptTime,ArrTime,flightNrSf,segment,DayOfWeekDpt,WeekOfMonthDpt,MonthDpt,...,QuarterArr,YearArr,IsWeekendArr,Holiday,actual,predictions,fltNr,opsSuffix,SegOrig,SegDest
44939,44319,F,2017-03-27,20,22,10J,12_23,0,5,3,...,1,2017,0,0,0,0,10,J,MOWKZ,RZN1
44940,44319,C,2017-03-27,20,22,10J,12_23,0,5,3,...,1,2017,0,0,0,1,10,J,MOWKZ,RZN1
44941,44319,B,2017-03-27,20,22,10J,12_23,0,5,3,...,1,2017,0,0,0,4,10,J,MOWKZ,RZN1
44942,44319,Y,2017-03-27,20,22,10J,12_23,0,5,3,...,1,2017,0,0,3,1,10,J,MOWKZ,RZN1
44943,44319,F,2017-03-27,20,3,10J,12_25,0,5,3,...,1,2017,0,0,0,0,10,J,MOWKZ,RUZVK
44944,44319,C,2017-03-27,20,3,10J,12_25,0,5,3,...,1,2017,0,0,1,1,10,J,MOWKZ,RUZVK
44945,44319,B,2017-03-27,20,3,10J,12_25,0,5,3,...,1,2017,0,0,1,0,10,J,MOWKZ,RUZVK
44946,44319,Y,2017-03-27,20,3,10J,12_25,0,5,3,...,1,2017,0,0,1,1,10,J,MOWKZ,RUZVK
44947,44319,F,2017-03-27,20,7,10J,12_58,0,5,3,...,1,2017,0,0,2,0,10,J,MOWKZ,SYZR1
44948,44319,C,2017-03-27,20,7,10J,12_58,0,5,3,...,1,2017,0,0,11,9,10,J,MOWKZ,SYZR1


In [201]:
import datetime

In [209]:
# 
whole_res.rename(columns={'flightNrSf':'FltNr', 'Date':'fltNrDptDt','resHoldTotal':'actual'}, inplace = True)

In [211]:
db = MySQLdb.connect(host="194.247.15.117", 
                     port=7806,
                     user="bar",
                     passwd="PASSWORD",
                     db="Inventory")

ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

for r in whole_res.iterrows():
    db.autocommit(True) 
    cur = db.cursor()
    proccess_flight(r)
    cur.close()

KeyboardInterrupt: 